<a href="https://colab.research.google.com/github/Ritesh313/Concentration-determination/blob/master/Concentration_determination_of_students_in_a_Classroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This line of the code is to remove anything from the colab directory
#!rm -rf /content/*

In [ ]:
#This is the code to import data from google drive
#Source - https://github.com/ndrplz/google-drive-downloader
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd
#I imported the zip folder containing all the files I need in the code.
gdd.download_file_from_google_drive(file_id='18SqzsPjkhoTpf6F-cHl4OhLQBXVjIsCj',
                                    dest_path='/content/data.zip',
                                    unzip=True)


Unzipping...Done.


In [ ]:
#This is the section to divide the video into frames.
import cv2 #Used to extract frames from video
import os #Used to create directories
frames='frames'
if not os.path.exists(frames):
  os.makedirs(frames) #This will create a folder named frames in the content directory of colab

#Path to video file imported in the above section
path = '/content/frames'
vid = cv2.VideoCapture('/content/Project files/VID_20200426_122440.mp4') 

# success variable checks whether frames were extracted 
success = 1
while success: 
  # vid is a cv2 object so, can be used to read frame 
  # function extract frames 
  success, image = vid.read()
  if(success==False):
    break
  timestamps = vid.get(cv2.CAP_PROP_POS_MSEC)/1000 #cv2.CAP_PROP_POS_MSEC -  This is the function to find the timestamp (in ms) of the frame.
  cv2.imwrite(os.path.join(path, "%d.jpg" % timestamps), image) 

In [ ]:
#Source: https://github.com/omar178/Emotion-recognition.git
#I used the pretrianed model from the above link
#But I changed the code to perform experiments on which emotion falls under focused and which under distracted. 
#Also, I changed the code to insert the data into a csv file
!git clone https://github.com/omar178/Emotion-recognition.git
from natsort import natsorted, ns
from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np
import glob
from google.colab.patches import cv2_imshow
import csv
import os

# These are the in-built models
detection_model_path = '/content/Emotion-recognition/haarcascade_files/haarcascade_frontalface_default.xml'
emotion_model_path = '/content/Emotion-recognition/models/_mini_XCEPTION.102-0.66.hdf5'

face_detection = cv2.CascadeClassifier(detection_model_path) 
emotion_classifier = load_model(emotion_model_path, compile=False) # Loading the model

EMOTIONS = ["angry" ,"disgust","confused", "happy", "gloomy", "surprised","neutral"] #These are the total 7 emotions
Distracted =["angry" ,"disgust","confused","gloomy", "surprised"] #The 5 emotions which I found that fall under Distracted by experimentation
Concentrated=["happy","neutral"] #These 2 emotions fall under Concentrated

with open('Concentration_log.csv', 'w') as file1: #This is for the csv file
  csv_writer=csv.writer(file1)
  csv_writer.writerow(["Timestamp(seconds)","Facial Expression","Drowsy","Concentration"]) #This writes the header in the csv file
  for path1 in natsorted(glob.glob('/content/frames/*.jpg'), key=lambda y: y.lower()): #Here I have used natsort to naturally sort the frames in ascending order.
    frame = cv2.imread(path1)
    #Reading the frame
    frame = imutils.resize(frame,width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    #The above line extracts faces from the frame
    for face in (faces):
      canvas = np.zeros((250, 300, 3), dtype="uint8") #This is for creating the canvas you see in the output image.
      frameClone = frame.copy()
      (fX, fY, fW, fH) = face
      # Extract the Regions Of Interest of the face from the grayscale image
      # Use the ROI for classification via the CNN
      roi = gray[fY:fY + fH, fX:fX + fW]
      roi = cv2.resize(roi, (64, 64))
      roi = roi.astype("float") / 255.0
      roi = img_to_array(roi)
      roi = np.expand_dims(roi, axis=0)
      preds = emotion_classifier.predict(roi)[0]
      emotion_probability = np.max(preds)
      label = EMOTIONS[preds.argmax()] #This is the final emotion prediction

      #Below I am extracting the timetamp from the name of the image.
      time=os.path.splitext(os.path.basename(path1))[0]
      #writing to csv_file
      if(label in Concentrated):
        state_final="Concentrated"
      else:
        state_final="Distracted"
      csv_writer.writerow([time,label,"NO",state_final])
      for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
        #Text to be printed
        text = "{}: {:.2f}%".format(emotion, prob * 100)
        # Drawing the Emotion + Probability index
        w = int(prob * 300)
        cv2.rectangle(canvas, (7, (i * 35) + 5),(w, (i * 35) + 35), (0, 0, 255), -1)
        cv2.putText(canvas, text, (10, (i * 35) + 23),cv2.FONT_HERSHEY_SIMPLEX, 0.45,(255, 255, 255), 2)
        cv2.putText(frameClone, label, (fX, fY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 2)
      cv2_imshow(frameClone) #This is for displaying the final image
      cv2_imshow(canvas)

In [ ]:
#Source: https://www.pyimagesearch.com/2017/05/08/drowsiness-detection-opencv/
#I used the pretrianed model from the above link
#But I performed experiments to determine the threhold value for the aspect ratio. 
#Also, I changed the code to insert the data into a csv file

from scipy.spatial import distance
from imutils import face_utils
import numpy as np
import time
import dlib
import cv2
import imutils
import glob
from google.colab.patches import cv2_imshow
from natsort import natsorted, ns

#Minimum threshold of eye aspect ratio below which alarm is triggerd
EYE_ASPECT_RATIO_THRESHOLD = 0.2

#Load face cascade which will be used to draw a rectangle around detected faces.
face_cascade = cv2.CascadeClassifier("/content/Project files/haarcascade_frontalface_default.xml")

#This function calculates and return eye aspect ratio
def eye_aspect_ratio(eye):
  A = distance.euclidean(eye[1], eye[5])
  B = distance.euclidean(eye[2], eye[4])
  C = distance.euclidean(eye[0], eye[3])

  ear = (A+B) / (2*C)
  return ear

#Load face detector and predictor, uses dlib shape predictor file
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/Project files/shape_predictor_68_face_landmarks.dat")

#Extract indexes of facial landmarks for the left and right eye
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

for path1 in natsorted(glob.glob('/content/frames/*.jpg'), key=lambda y: y.lower()):
  #Read each frame and flip it, and convert to grayscale

  frame=cv2.imread(path1)
  frame = imutils.resize(frame, width=300)
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  
  #Detect facial points through detector function
  faces = detector(gray, 0)

  #Detect faces through haarcascade_frontalface_default.xml
  face_rectangle = face_cascade.detectMultiScale(gray, 1.3, 5)

  #Draw rectangle around each face detected
  for (x,y,w,h) in face_rectangle:
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

  #Detect facial points
  for face in faces:
    shape = predictor(gray, face)
    shape = face_utils.shape_to_np(shape)

    #Get array of coordinates of leftEye and rightEye
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]

    #Calculate aspect ratio of both eyes
    leftEyeAspectRatio = eye_aspect_ratio(leftEye)
    rightEyeAspectRatio = eye_aspect_ratio(rightEye)

    eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2

    #Use hull to remove convex contour discrepencies and draw eye shape around eyes
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

    #Detect if eye aspect ratio is less than threshold
    if(eyeAspectRatio < EYE_ASPECT_RATIO_THRESHOLD):
      cv2_imshow(frame)
      time=int(os.path.splitext(os.path.basename(path1))[0])
      ### This is for editing the csv file
      f = open('/content/Concentration_log.csv', 'r')
      reader = csv.reader(f)
      mylist = list(reader)
      f.close()
      mylist[time+1][2]='YES'
      my_new_list = open('/content/Concentration_log.csv', 'w', newline = '')
      csv_writer = csv.writer(my_new_list)
      csv_writer.writerows(mylist)
      my_new_list.close()
      ### csv file edited
      print("Drowsy")
      
    else:
      continue
        

In [ ]:
#This piece of code is to use the drowsiness determinator to find the concentration
#Facial expression is not the only reliable pointer in case the person is drowsy. 
#Hence, the cocentration will be changed if the person is drowsy

f = open('/content/Concentration_log.csv', 'r')
reader = csv.reader(f)
mylist = list(reader)

for i in range(1,(len(mylist))):
  if(mylist[i][2]=="YES"):
    mylist[i][3]="Distracted"
f.close()

my_new_list = open('/content/Concentration_log.csv', 'w', newline = '')
csv_writer = csv.writer(my_new_list)
csv_writer.writerows(mylist)
my_new_list.close()
